# Who Wrote the Gospels?
The Christian Bible consists of 66 books. Four of these books (Matthew, Mark, Luke, John) tell the life of Jesus; these 4 books are collectively known as the Gospels.

In this activity, you will use data science to rediscover a fact about the Gospels that have long been known to religion scholars, without reading the text!

# Exercise 0. Reading in the Data
The text of the four gospels are stored on the web in four files:

Matthew: https://datasci112.stanford.edu/data/gospels/matthew.txt
Mark: https://datasci112.stanford.edu/data/gospels/mark.txt
Luke: https://datasci112.stanford.edu/data/gospels/luke.txt
John: https://datasci112.stanford.edu/data/gospels/john.txt
Read in the four texts into a list called corpus.

In [1]:
dir = "https://datasci112.stanford.edu/data/gospels/"
gospel_files = ["matthew.txt", "mark.txt", "luke.txt", "john.txt"]

In [2]:
# YOUR CODE HERE
import requests

corpus = []

for filename in gospel_files:
    res = requests.get(dir + filename , "r")
    corpus.append(res.text)

In [3]:
corpus

[' The book of the generation of Jesus Christ, the son of David, the\nson of Abraham.\n\n Abraham begat Isaac; and Isaac begat Jacob; and Jacob begat Judas\nand his brethren;  And Judas begat Phares and Zara of Thamar; and\nPhares begat Esrom; and Esrom begat Aram;  And Aram begat Aminadab;\nand Aminadab begat Naasson; and Naasson begat Salmon;  And Salmon\nbegat Booz of Rachab; and Booz begat Obed of Ruth; and Obed begat\nJesse;  And Jesse begat David the king; and David the king begat\nSolomon of her that had been the wife of Urias;  And Solomon begat\nRoboam; and Roboam begat Abia; and Abia begat Asa;  And Asa begat\nJosaphat; and Josaphat begat Joram; and Joram begat Ozias;  And\nOzias begat Joatham; and Joatham begat Achaz; and Achaz begat Ezekias;\n And Ezekias begat Manasses; and Manasses begat Amon; and Amon\nbegat Josias;  And Josias begat Jechonias and his brethren, about\nthe time they were carried away to Babylon:  And after they were\nbrought to Babylon, Jechonias begat Sa

# Exercise 1. Term-Frequency Matrix
Construct the term-frequency matrix for this corpus, and calculate the Euclidean distances between all pairs of gospels. Which two gospels are most similar? Most different?

In [4]:
# YOUR CODE HERE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
import numpy as np

vec = CountVectorizer()
vec.fit(corpus)
tf = vec.transform(corpus)

distance_matrix = pairwise_distances(tf)
distance_matrix


D = np.array(distance_matrix)
D_no_diag = D.copy()
np.fill_diagonal(D_no_diag, np.inf)

nearest_idx = D_no_diag.argmin(axis=1)
nearest_dist = D_no_diag.min(axis=1)

i, j = np.unravel_index(D_no_diag.argmin(), D_no_diag.shape)
print("\nMost similar pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])


i, j = np.unravel_index(D.argmax(), D_no_diag.shape)
print("\nMost different pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])

print("\n", distance_matrix)



Most similar pair overall: matthew.txt <-> luke.txt distance= 634.598298138279

Most different pair overall: luke.txt <-> john.txt distance= 1332.1801679953053

 [[   0.          911.40331358  634.59829814  965.09947674]
 [ 911.40331358    0.         1222.63445068  765.52139095]
 [ 634.59829814 1222.63445068    0.         1332.180168  ]
 [ 965.09947674  765.52139095 1332.180168      0.        ]]


YOUR CONCLUSION HERE

When we look the distance matrix we can see the minimum distance is 634.59829814 (with ignoring self distances - diagonals) and the maximum distance is 1332.180168. 

Minimum distance is between indices 0 - 2, matthew and luke (From the gospel_files)
Maximum distance is between indices 2 - 3, luke and john (From the gospel_files)

Results are also printed at the cell distances are calculated.

# Exercise 2. Vector Space Model
In the vector space model, we treat the documents as vectors instead of points. To measure the distance between two vectors, we use cosine distance.

Calculate the cosine distances between all pairs of gospels. What do you notice now? How does your conclusion compare with the one you obtained above?

In [5]:
# YOUR CODE HERE

cosine_distances = pairwise_distances(tf, metric="cosine")

D = np.array(cosine_distances)
D_no_diag = D.copy()
np.fill_diagonal(D_no_diag, np.inf)

nearest_idx = D_no_diag.argmin(axis=1)
nearest_dist = D_no_diag.min(axis=1)

i, j = np.unravel_index(D_no_diag.argmin(), D_no_diag.shape)
print("\nMost similar pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])


i, j = np.unravel_index(D.argmax(), D_no_diag.shape)
print("\nMost different pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])

print("\n", cosine_distances)




Most similar pair overall: mark.txt <-> luke.txt distance= 0.014194692754025962

Most different pair overall: mark.txt <-> john.txt distance= 0.07263467507972732

 [[0.         0.02320875 0.01549838 0.04716941]
 [0.02320875 0.         0.01419469 0.07263468]
 [0.01549838 0.01419469 0.         0.05751567]
 [0.04716941 0.07263468 0.05751567 0.        ]]


YOUR CONCLUSION HERE


This time pairs are different.

Closest pair is mark and luke

Farthest pair is mark and jhon

# Exercise 2a. (optional)
Notice that Euclidean distance and cosine distance reached different conclusions about which Gospels were most similar. This is because longer documents have more words; Euclidean distance is sensitive to this, while cosine distance is not.

In fact, if we just normalize the word counts in each row before calculating Euclidean distance, then Euclidean distance and cosine distance will reach the exact same conclusion!

Normalizing a vector means scaling the vector so that its length is 1.
To do this, we divide the vector by its length.
v ← v / ||v||. 
This can be done in Pandas or using sklearn.preprocessing.Normalizer.

In [6]:
# YOUR CODE HERE
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances
import numpy as np

tf_norm = normalize(tf, norm="l2", axis=1)

euclid_norm = pairwise_distances(tf_norm, metric="euclidean")
cosine_dist = pairwise_distances(tf, metric="cosine")

D = np.array(euclid_norm)
D_no_diag = D.copy()
np.fill_diagonal(D_no_diag, np.inf)

i, j = np.unravel_index(D_no_diag.argmin(), D_no_diag.shape)
print("Most similar (euclidean on normalized):", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])

print("\nEuclidean (normalized):\n", euclid_norm)
print("\nCosine distances:\n", cosine_dist)

Most similar (euclidean on normalized): mark.txt <-> luke.txt distance= 0.16849149980951603

Euclidean (normalized):
 [[0.         0.21544719 0.17605894 0.30714624]
 [0.21544719 0.         0.1684915  0.38114217]
 [0.17605894 0.1684915  0.         0.33916269]
 [0.30714624 0.38114217 0.33916269 0.        ]]

Cosine distances:
 [[0.         0.02320875 0.01549838 0.04716941]
 [0.02320875 0.         0.01419469 0.07263468]
 [0.01549838 0.01419469 0.         0.05751567]
 [0.04716941 0.07263468 0.05751567 0.        ]]


In math courses, you will learn how to show why normalizing the vectors makes Euclidean distance agree with cosine distance.

In fact, there's a very simple relationship between the Euclidean distance between the normalized vectors and the cosine distance:
Euclidean distance (between the normalized vectors)=2⋅cosine distance−−−−−−−−−−−−−−√. 

 see if you can show this!

In [7]:
print(euclid_norm)

print(np.sqrt(2 * cosine_dist))



[[0.         0.21544719 0.17605894 0.30714624]
 [0.21544719 0.         0.1684915  0.38114217]
 [0.17605894 0.1684915  0.         0.33916269]
 [0.30714624 0.38114217 0.33916269 0.        ]]
[[0.         0.21544719 0.17605894 0.30714624]
 [0.21544719 0.         0.1684915  0.38114217]
 [0.17605894 0.1684915  0.         0.33916269]
 [0.30714624 0.38114217 0.33916269 0.        ]]


# Exercise 3. tf-idf
We should probably upweight rare words and downweight common words. Construct the tf-idf matrix, and calculate the cosine distances between all pairs of gospels. Do your conclusions change compared to Exercise 2?




In [8]:
# YOUR CODE HERE
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(smooth_idf=False, norm=None)

vec.fit(corpus)

tfidf_matrix = vec.transform(corpus)

tfidf_distances = pairwise_distances(tfidf_matrix, metric="cosine")

tfidf_distances


D = np.array(tfidf_distances)
D_no_diag = D.copy()
np.fill_diagonal(D_no_diag, np.inf)

nearest_idx = D_no_diag.argmin(axis=1)
nearest_dist = D_no_diag.min(axis=1)

i, j = np.unravel_index(D_no_diag.argmin(), D_no_diag.shape)
print("\nMost similar pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])


i, j = np.unravel_index(D.argmax(), D_no_diag.shape)
print("\nMost different pair overall:", gospel_files[i], "<->", gospel_files[j], "distance=", D_no_diag[i, j])

print("\n", tfidf_distances)


Most similar pair overall: mark.txt <-> luke.txt distance= 0.015221452441373295

Most different pair overall: mark.txt <-> john.txt distance= 0.07439184085509498

 [[0.         0.02475781 0.01713163 0.04968209]
 [0.02475781 0.         0.01522145 0.07439184]
 [0.01713163 0.01522145 0.         0.05928521]
 [0.04968209 0.07439184 0.05928521 0.        ]]


YOUR CONCLUSION HERE

Reached same results with Exercise 2.

Further Reading
You have just discovered a phenomenon known to critical Biblical scholars as the Synoptic (https://en.wikipedia.org/wiki/Synoptic_Gospels) problem!